## Trabajo final

Obtencion de los datos de criptomonedas.

Usamos el exchanges de cryptos HitBTC, utilizamos la **API** de **demo.hitbtc.com**


Acerca de esta API (documentacion):
https://api.demo.hitbtc.com/#about-hitbtc-api

Vamos a trabajar con el mercado spot (contado) y este exchange opera con casi 400 crytos diferentes.

In [1]:
import requests
import pandas as pd

Listado de monedas

Definimos una funcion para obterner el listado de monedas con las cuales se puede operar.

In [2]:
def listaCryptos():
    url= 'https://api.hitbtc.com/api/2/public/currency'
    r= requests.get(url)
    js= r.json()
    df= pd.DataFrame(js).set_index('id')
    return df

In [3]:
cryptos= listaCryptos()
cryptos.loc[:,['fullName']]

,fullName
id,
DDF,DDF
ZRX,0x Protocol
ACO,ACO
ACT,Achain
ADX,AdEx
...,...
BAND,Band Protocol
TVK,Terra Virtua Kolect
STPT,Standard Tokenization Protocol


In [4]:
cryptos.reset_index(inplace=True)
cryptos

,id,fullName,crypto,payinEnabled,payinPaymentId,payinConfirmations,payoutEnabled,payoutIsPaymentId,transferEnabled,delisted,payoutFee,precisionPayout,precisionTransfer,lowProcessingTime,highProcessingTime,avgProcessingTime
0,DDF,DDF,True,False,False,9,True,False,True,False,646.000000000000,8,8,NaN,NaN,NaN
1,ZRX,0x Protocol,True,True,False,9,True,False,True,False,29.367000000000,8,8,1.94,29098.965,2977.216880000001
2,ACO,ACO,True,False,False,9,True,False,True,False,36929.00000000000,8,8,NaN,NaN,NaN
3,ACT,Achain,True,True,True,20,True,True,True,False,433.010000000000,8,8,1.659,8.898,5.892666666666667
4,ADX,AdEx,True,False,False,9,False,False,True,False,51.670000000000,4,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,BAND,Band Protocol,True,True,False,9,True,False,True,False,3.500000000000,8,8,232531.06,232531.06,232531.05999999997
765,TVK,Terra Virtua Kolect,True,True,False,9,True,False,True,False,48.000000000000,8,8,69.918,102.396,86.157
766,STPT,Standard Tokenization Protocol,True,True,False,9,True,False,True,False,570.000000000000,8,8,821.786,862.767,842.2764999999999
767,OCEAN,Ocean Protocol,True,True,False,9,True,False,True,False,24.000000000000,8,8,343.417,391.789,367.603


In [5]:
mask= (cryptos.id == 'BTC') |(cryptos.id == 'ETH')| (cryptos.id == 'XRP')
cryptos[mask]

,id,fullName,crypto,payinEnabled,payinPaymentId,payinConfirmations,payoutEnabled,payoutIsPaymentId,transferEnabled,delisted,payoutFee,precisionPayout,precisionTransfer,lowProcessingTime,highProcessingTime,avgProcessingTime
36,BTC,Bitcoin,True,True,False,1,True,False,True,False,0.000900000000,8,8,22.541,323.988,127.08971264367814
131,ETH,Ethereum,True,True,False,9,True,False,True,False,0.011600000000,18,12,2.134,503.337,17.321154639175255
269,XRP,Ripple,True,True,True,20,True,True,True,False,12.790000000000,6,6,1.198,3073.293,75.83144897959185


### Obtenemos los precios historicos para una criptomoneda

Los valores que acepta la API para los periodos:

<li>M1, 1 minuto
<li>M3, 3 minutos
<li>M5, 5 minutos
<li>M15, 15 minutos
<li>M30, 30 minutos
<li>H1, 1 hora
<li>H4, 4 horas
<li>D1, 1 dia
<li>D7, 7 dias
<li>1M, 1 mes

In [6]:
# funcion para traer las velas de la criptomoneda a traves de la API del exchange
# sort values= 'DESC' o 'ASC'
def getVelas(crypto, periodo, desde, hasta, limite=1000, sort='ASC'):
    url= "https://api.hitbtc.com/api/2/public/candles"
    p= {'limit':limite, 'symbols': crypto, 'from': desde, 'till': hasta,
        'period':periodo,'sort': sort}
    r= requests.get(url, params= p)
    js= r.json()[crypto]
    df= pd.DataFrame(js)
    df['crypto']= crypto
    return df

In [7]:
# bajar del exchange los datos de las criptomonedas seleccionadas
lista_cryptos= ['BTCUSD','ETHUSD']

# el desde es a partir de que fecha trae las velas
# el hasta es la fecha hasta la que se trae los valores hasta el valor limite que por
# default que es 1.000. Mil es le maximo de datos que trae esta demo API
i= 0
for c in lista_cryptos:
    if i== 0:
        datos= getVelas(c, 'D1', desde='2018-03-15', hasta='2021-03-16')
        i =  i + 1
    else:
        datos= datos.append(getVelas(c, 'D1', desde='2018-03-15', hasta='2021-03-15'),
                    ignore_index=True)

In [8]:
datos

,timestamp,open,close,min,max,volume,volumeQuote,crypto
0,2018-03-15T00:00:00.000Z,8601,8409.73,8066.61,8751.75,7166.70,60423962.8691,BTCUSD
1,2018-03-16T00:00:00.000Z,8411.31,8503.51,8124.39,8848.95,5823.86,49557058.6743,BTCUSD
2,2018-03-17T00:00:00.000Z,8503.21,8143.04,8050.27,8600,6504.02,54082510.1424,BTCUSD
3,2018-03-18T00:00:00.000Z,8144.99,8520.22,7715,8600,7165.17,57516450.5240,BTCUSD
4,2018-03-19T00:00:00.000Z,8526.54,8861.91,8375.7,8900,10400.01,89675569.1852,BTCUSD
...,...,...,...,...,...,...,...,...
1995,2020-12-04T00:00:00.000Z,615.987,566.626,565,619.692,124498.6007,74799350.8174249,ETHUSD
1996,2020-12-05T00:00:00.000Z,566.72,596.485,559.164,596.836,15290.4828,8923454.1654934,ETHUSD
1997,2020-12-06T00:00:00.000Z,596.742,601.294,583.397,607.369,13366.5558,7969615.2948912,ETHUSD
1998,2020-12-07T00:00:00.000Z,601.427,591.326,584.078,602.827,68615.7469,40665085.8848745,ETHUSD


In [10]:
datos_path= '../Datos/'
file= 'velas_cryptos'
extencion= '.csv'
datos.to_csv(datos_path+file+extencion, sep=',', index=False)